***DataSpark: Illuminating Insights for Global Electronics***

In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\USER\.virtualenvs\pythonProject\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
# loading customers.csv

import pandas as pd

file_path = r"C:/Users/USER/Desktop/DataSpark_Illuminating_Insights_for_Global_Electronics/Customers.csv"

# Method 1: Try common encodings automatically
encodings = ['utf-8', 'latin1', 'cp1252', 'ISO-8859-1']

for encoding in encodings:
    try:
        df_Customers = pd.read_csv(file_path, encoding=encoding)
        print(f"Successfully loaded with {encoding} encoding!")
        break
    except UnicodeDecodeError:
        continue

# Method 2: Manual encoding specification (if automatic fails)
# df_Customers = pd.read_csv(file_path, encoding='latin1')  # Most reliable fallback

# Verify the DataFrame
print("\nFirst 5 rows of df_Customers:")
print(df_Customers.head())

print("\nDataFrame info:")
print(df_Customers.info())

Successfully loaded with latin1 encoding!

First 5 rows of df_Customers:
   CustomerKey  Gender               Name            City State Code  \
0          301  Female      Lilly Harding  WANDEARAH EAST         SA   
1          325  Female       Madison Hull      MOUNT BUDD         WA   
2          554  Female      Claire Ferres       WINJALLOK        VIC   
3          786    Male  Jai Poltpalingada    MIDDLE RIVER         SA   
4         1042    Male    Aidan Pankhurst   TAWONGA SOUTH        VIC   

               State Zip Code    Country  Continent    Birthday  
0    South Australia     5523  Australia  Australia    7/3/1939  
1  Western Australia     6522  Australia  Australia   9/27/1979  
2           Victoria     3380  Australia  Australia   5/26/1947  
3    South Australia     5223  Australia  Australia   9/17/1957  
4           Victoria     3698  Australia  Australia  11/19/1965  

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15266 entries, 0 to 15265
Data 

In [3]:
df_Customers.dtypes


CustomerKey     int64
Gender         object
Name           object
City           object
State Code     object
State          object
Zip Code       object
Country        object
Continent      object
Birthday       object
dtype: object

In [4]:
# Convert categorical/text columns to string
df_Customers["Gender"] = df_Customers["Gender"].astype("string")
df_Customers["Name"] = df_Customers["Name"].astype("string")
df_Customers["City"] = df_Customers["City"].astype("string")
df_Customers["State"] = df_Customers["State"].astype("string")
df_Customers["Country"] = df_Customers["Country"].astype("string")
df_Customers["Continent"] = df_Customers["Continent"].astype("string")
df_Customers["State Code"] = df_Customers["State Code"].astype("string")
df_Customers["Zip Code"] = df_Customers["Zip Code"].astype("string")

# Convert Birthday to datetime format
df_Customers["Birthday"] = pd.to_datetime(df_Customers["Birthday"], format="%m/%d/%Y", errors="coerce")

# Verify the changes
print(df_Customers.dtypes)


CustomerKey             int64
Gender         string[python]
Name           string[python]
City           string[python]
State Code     string[python]
State          string[python]
Zip Code       string[python]
Country        string[python]
Continent      string[python]
Birthday       datetime64[ns]
dtype: object


In [5]:
df_Customers_cleaned = df_Customers

In [16]:
# Verify the changes
print(df_Customers_cleaned.dtypes)

CustomerKey     int64
Gender         object
Name           object
City           object
State Code     object
State          object
Zip Code       object
Country        object
Continent      object
Birthday       object
dtype: object


In [7]:
df_Customers_cleaned.to_csv('df_Customers_cleaned.csv', index=False)

In [8]:
df_Customers_cleaned = pd.read_csv('df_Customers_cleaned.csv')

In [9]:
df_Customers_cleaned.head()

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,1939-07-03
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,1979-09-27
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,1947-05-26
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,1957-09-17
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,1965-11-19


In [2]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\USER\.virtualenvs\pythonProject\Scripts\python.exe -m pip install --upgrade pip' command.


In [12]:
pip install sqlalchemy

  Using cached greenlet-3.1.1-cp310-cp310-win_amd64.whl (298 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\USER\.virtualenvs\pythonProject\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
#push into pg sql , Customers_cleaned data _ final
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from datetime import datetime
from sqlalchemy import create_engine

try:
    # Connect to PostgreSQL
    connection = psycopg2.connect(
        host="localhost",
        port="5432",
        database="Global_Electronics_Data",
        user="postgres",
        password="sample12"
    )
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cursor = connection.cursor()

    # Check if table exists
    cursor.execute("""
        SELECT EXISTS (
            SELECT FROM information_schema.tables 
            WHERE table_name = 'customers'
        );
    """)
    table_exists = cursor.fetchone()[0]

    if not table_exists:
        print("Table 'customers' does not exist. Creating it now...")
        cursor.execute("""
            CREATE TABLE customers (
                CustomerKey INT PRIMARY KEY,
                Gender VARCHAR(255),
                Name TEXT,
                City VARCHAR(255),
                State_Code VARCHAR(255),
                State VARCHAR(255),
                Zip_Code VARCHAR(255),  -- Changed from INT to VARCHAR
                Country VARCHAR(255),
                Continent VARCHAR(255),
                Birthday DATE
            );
        """)
        print("Table created successfully.")
    
    # Insert data
    for index, row in df_Customers_cleaned.iterrows():
        cursor.execute("""
            INSERT INTO customers (CustomerKey, Gender, Name, City, State_Code, 
            State, Zip_Code, Country, Continent, Birthday) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['CustomerKey'], row['Gender'], row['Name'], row['City'],
            row['State Code'], row['State'], str(row['Zip Code']),  # Ensure Zip Code is string
            row['Country'], row['Continent'], row['Birthday']
        ))

    connection.commit()
    print("Data inserted successfully.")

except Exception as e:
    print(f"An error occurred while interacting with the database: {e}")

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()
        

Table 'customers' does not exist. Creating it now...
Table created successfully.
Data inserted successfully.


In [17]:
print(df_Customers_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15266 entries, 0 to 15265
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CustomerKey  15266 non-null  int64 
 1   Gender       15266 non-null  object
 2   Name         15266 non-null  object
 3   City         15266 non-null  object
 4   State Code   15256 non-null  object
 5   State        15266 non-null  object
 6   Zip Code     15266 non-null  object
 7   Country      15266 non-null  object
 8   Continent    15266 non-null  object
 9   Birthday     15266 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.2+ MB
None


In [ ]:
#push into pg sql , Customers_cleaned data
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from datetime import datetime
from sqlalchemy import create_engine

try:
    # Connect to PostgreSQL
    connection = psycopg2.connect(
        host="localhost",
        port="5432",
        database="Global_Electronics_Data",
        user="postgres",
        password="sample12"
    )
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cursor = connection.cursor()

    # Check if table exists
    cursor.execute("""
        SELECT EXISTS (
            SELECT FROM information_schema.tables 
            WHERE table_name = 'Customers'
        );
    """)
    table_exists = cursor.fetchone()[0]

    if not table_exists:
        print("Table 'bus_routes' does not exist. Creating it now...")
        cursor.execute("""
            CREATE TABLE bus_routes (
                CustomerKey PRIMARY KEY,
                Gender VARCHAR(255),
                Name TEXT,
                City VARCHAR(255),
                State_Code VARCHAR(255),
                State VARCHAR(255),
                Zip_Code INT,
                Country VARCHAR(255),
                Continent VARCHAR(255),
                Birthday ,
            );
        """)
        print("Table created successfully.")
    

    # Insert data
    for index,row in df_Customers_cleaned.iterrows():
        cursor.execute("""
            INSERT INTO bus_routes (CustomerKey, Gender, Name,City, State_Code, 
            State, Zip_Code, Country, Continent,Birthday) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['CustomerKey'], row['Gender'], row['Name'], row['City'],
            row['State Code'], row['State'],row['Zip Code'],row['Country'],row['Continent'],row['Birthday']
        ))

    connection.commit()
    print("Data inserted successfully.")

except Exception as e:
    print(f"An error occurred while interacting with the database: {e}")

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()

In [36]:
print(df_Customers["State Code"].unique())


['SA' 'WA' 'VIC' 'QLD' 'NT' 'NSW' 'TAS' 'ACT' 'BC' 'QC' 'ON' 'AB' 'NS'
 'SK' 'NU' 'PE' 'MB' 'NL' 'YT' 'NB' 'BB' 'RP' 'BY' 'BW' 'NW' 'NI' 'ST'
 'MV' 'SN' 'TH' 'BE' 'HE' 'SL' 'SH' 'HB' 'HH' 'RA' 'IL' 'GY' 'AL' 'AQ'
 'CA' 'NP' 'GD' 'FC' 'PA' 'PC' 'LI' 'MP' 'PI' 'HN' 'AU' 'PL' 'LO' 'BN'
 'CE' 'LN' 'MQ' 'BO' 'BR' 'CO' 'MY' 'RC' 'VI' 'FE' 'RM' 'AG' 'IM' 'MI'
 'PR' 'BG' 'RG' 'PN' 'SV' 'LU' 'CN' 'TN' 'LE' 'PD' 'BI' 'CH' 'GE' 'TO'
 'VV' 'CZ' 'AN' 'PG' 'LT' 'BL' 'TV' 'PV' 'MN' 'VA' 'PT' 'SI' 'MS' 'CT'
 'BS' 'SS' 'RO' 'CR' 'FI' 'GR' 'IS' 'SO' 'VE' 'OR' 'ME' 'VR' 'CS' 'BZ'
 'NO' 'AV' 'TA' 'VC' 'GO' 'MO' 'FR' 'FG' 'TE' 'BA' 'UD' 'AP' 'TP' 'RE'
 'PO' nan 'AT' 'SR' 'RI' 'TS' 'KR' 'MT' 'PZ' 'MC' 'VT' 'AR' 'AO' 'CB' 'LC'
 'SP' 'RN' 'FO' 'TR' 'UT' 'NH' 'DR' 'ZH' 'FL' 'OV' 'ZE' 'Falkirk'
 'Ceredigion' 'North East Lincolnshire' 'Aberdeenshire' 'York'
 'Pembrokeshire' 'Leicester' 'Highland' 'Tendring' 'Horsham' 'Newport'
 'Bristol' 'Newark and Sherwood' 'Argyllshire' 'Lincoln' 'Tamworth'
 'Fylde' 'Lewes' '

In [39]:
print(df_Customers["Zip Code"])

0         5523
1         6522
2         3380
3         5223
4         3698
         ...  
15261    77017
15262    22101
15263    28405
15264    92501
15265    48302
Name: Zip Code, Length: 15266, dtype: object


In [41]:
print(df_Customers.dtypes)


CustomerKey             int64
Gender         string[python]
Name           string[python]
City           string[python]
State Code     string[python]
State          string[python]
Zip Code       string[python]
Country        string[python]
Continent      string[python]
Birthday       datetime64[ns]
dtype: object


In [42]:
print(df_Customers.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15266 entries, 0 to 15265
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   CustomerKey  15266 non-null  int64         
 1   Gender       15266 non-null  string        
 2   Name         15266 non-null  string        
 3   City         15266 non-null  string        
 4   State Code   15256 non-null  string        
 5   State        15266 non-null  string        
 6   Zip Code     15266 non-null  string        
 7   Country      15266 non-null  string        
 8   Continent    15266 non-null  string        
 9   Birthday     15266 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), string(8)
memory usage: 1.2 MB
None


In [52]:
df_Customers.to_csv("customers_cleaned.csv", index=True)


In [53]:
import pandas as pd

# Load the cleaned CSV
df_customers_cleaned = pd.read_csv("customers_cleaned.csv")

# Display the entire DataFrame (if it's small)
print(df_customers_cleaned)  # Shows all rows without truncation


       Unnamed: 0  CustomerKey  Gender               Name  \
0               0          301  Female      Lilly Harding   
1               1          325  Female       Madison Hull   
2               2          554  Female      Claire Ferres   
3               3          786    Male  Jai Poltpalingada   
4               4         1042    Male    Aidan Pankhurst   
...           ...          ...     ...                ...   
15261       15261      2099600  Female     Denisa Duková   
15262       15262      2099618    Male   Justin Solórzano   
15263       15263      2099758    Male    Svend Petrussen   
15264       15264      2099862  Female       Lorenza Rush   
15265       15265      2099937    Male   Zygmunt Kaminski   

                      City State Code              State Zip Code  \
0           WANDEARAH EAST         SA    South Australia     5523   
1               MOUNT BUDD         WA  Western Australia     6522   
2                WINJALLOK        VIC           Victoria    

In [54]:
print(df_customers_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15266 entries, 0 to 15265
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   15266 non-null  int64 
 1   CustomerKey  15266 non-null  int64 
 2   Gender       15266 non-null  object
 3   Name         15266 non-null  object
 4   City         15266 non-null  object
 5   State Code   15256 non-null  object
 6   State        15266 non-null  object
 7   Zip Code     15266 non-null  object
 8   Country      15266 non-null  object
 9   Continent    15266 non-null  object
 10  Birthday     15266 non-null  object
dtypes: int64(2), object(9)
memory usage: 1.3+ MB
None


****Products****

In [39]:
# loading Products.csv

import pandas as pd

file_path = r"C:/Users/USER/Desktop/DataSpark_Illuminating_Insights_for_Global_Electronics/Products.csv"

# Method 1: Try common encodings automatically
encodings = ['utf-8', 'latin1', 'cp1252', 'ISO-8859-1']

for encoding in encodings:
    try:
        df_Products = pd.read_csv(file_path, encoding=encoding)
        print(f"Successfully loaded with {encoding} encoding!")
        break
    except UnicodeDecodeError:
        continue

# Method 2: Manual encoding specification (if automatic fails)
# df_Products = pd.read_csv(file_path, encoding='latin1')  # Most reliable fallback

# Verify the DataFrame
print("\nFirst 5 rows of df_Products:")
print(df_Products.head())

print("\nDataFrame info:")
print(df_Products.info())

Successfully loaded with utf-8 encoding!

First 5 rows of df_Products:
   ProductKey                         Product Name    Brand   Color  \
0           1  Contoso 512MB MP3 Player E51 Silver  Contoso  Silver   
1           2    Contoso 512MB MP3 Player E51 Blue  Contoso    Blue   
2           3     Contoso 1G MP3 Player E100 White  Contoso   White   
3           4    Contoso 2G MP3 Player E200 Silver  Contoso  Silver   
4           5       Contoso 2G MP3 Player E200 Red  Contoso     Red   

  Unit Cost USD Unit Price USD  SubcategoryKey Subcategory  CategoryKey  \
0        $6.62         $12.99              101     MP4&MP3            1   
1        $6.62         $12.99              101     MP4&MP3            1   
2        $7.40         $14.52              101     MP4&MP3            1   
3       $11.00         $21.57              101     MP4&MP3            1   
4       $11.00         $21.57              101     MP4&MP3            1   

  Category  
0    Audio  
1    Audio  
2    Audio  

In [41]:
# Convert categorical/text columns to string
df_Products["Product Name"] = df_Products["Product Name"].astype("string")
df_Products["Brand"] = df_Products["Brand"].astype("string")
df_Products["Color"] = df_Products["Color"].astype("string")
df_Products["Subcategory"] = df_Products["Subcategory"].astype("string")
df_Products["Category"] = df_Products["Category"].astype("string")

# Convert currency columns to numeric (removing $ sign first)
df_Products["Unit Cost USD"] = df_Products["Unit Cost USD"].replace('[\$,]', '', regex=True).astype(float)
df_Products["Unit Price USD"] = df_Products["Unit Price USD"].replace('[\$,]', '', regex=True).astype(float)

# Verify the changes

print(df_Products.dtypes)


ProductKey                 int64
Product Name      string[python]
Brand             string[python]
Color             string[python]
Unit Cost USD            float64
Unit Price USD           float64
SubcategoryKey             int64
Subcategory       string[python]
CategoryKey                int64
Category          string[python]
dtype: object


In [42]:
print(df_Products.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ProductKey      2517 non-null   int64  
 1   Product Name    2517 non-null   string 
 2   Brand           2517 non-null   string 
 3   Color           2517 non-null   string 
 4   Unit Cost USD   2517 non-null   float64
 5   Unit Price USD  2517 non-null   float64
 6   SubcategoryKey  2517 non-null   int64  
 7   Subcategory     2517 non-null   string 
 8   CategoryKey     2517 non-null   int64  
 9   Category        2517 non-null   string 
dtypes: float64(2), int64(3), string(5)
memory usage: 196.8 KB
None


In [43]:
Products_cleaned=df_Products


In [44]:
print(Products_cleaned.head())


   ProductKey                         Product Name    Brand   Color  \
0           1  Contoso 512MB MP3 Player E51 Silver  Contoso  Silver   
1           2    Contoso 512MB MP3 Player E51 Blue  Contoso    Blue   
2           3     Contoso 1G MP3 Player E100 White  Contoso   White   
3           4    Contoso 2G MP3 Player E200 Silver  Contoso  Silver   
4           5       Contoso 2G MP3 Player E200 Red  Contoso     Red   

   Unit Cost USD  Unit Price USD  SubcategoryKey Subcategory  CategoryKey  \
0           6.62           12.99             101     MP4&MP3            1   
1           6.62           12.99             101     MP4&MP3            1   
2           7.40           14.52             101     MP4&MP3            1   
3          11.00           21.57             101     MP4&MP3            1   
4          11.00           21.57             101     MP4&MP3            1   

  Category  
0    Audio  
1    Audio  
2    Audio  
3    Audio  
4    Audio  


In [45]:
print(Products_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ProductKey      2517 non-null   int64  
 1   Product Name    2517 non-null   string 
 2   Brand           2517 non-null   string 
 3   Color           2517 non-null   string 
 4   Unit Cost USD   2517 non-null   float64
 5   Unit Price USD  2517 non-null   float64
 6   SubcategoryKey  2517 non-null   int64  
 7   Subcategory     2517 non-null   string 
 8   CategoryKey     2517 non-null   int64  
 9   Category        2517 non-null   string 
dtypes: float64(2), int64(3), string(5)
memory usage: 196.8 KB
None


In [35]:
Products_cleaned.to_csv("Products_cleaned.csv", index=False)


In [36]:
import pandas as pd

# Load the cleaned CSV
Products_cleaned = pd.read_csv("Products_cleaned.csv")

# Display the entire DataFrame (if it's small)
print(Products_cleaned)  # Shows all rows without truncation


      ProductKey                                   Product Name    Brand  \
0              1            Contoso 512MB MP3 Player E51 Silver  Contoso   
1              2              Contoso 512MB MP3 Player E51 Blue  Contoso   
2              3               Contoso 1G MP3 Player E100 White  Contoso   
3              4              Contoso 2G MP3 Player E200 Silver  Contoso   
4              5                 Contoso 2G MP3 Player E200 Red  Contoso   
...          ...                                            ...      ...   
2512        2513    Contoso Bluetooth Active Headphones L15 Red  Contoso   
2513        2514  Contoso Bluetooth Active Headphones L15 White  Contoso   
2514        2515             Contoso In-Line Coupler E180 White  Contoso   
2515        2516             Contoso In-Line Coupler E180 Black  Contoso   
2516        2517            Contoso In-Line Coupler E180 Silver  Contoso   

       Color  Unit Cost USD  Unit Price USD  SubcategoryKey  \
0     Silver           6

In [46]:
print(Products_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ProductKey      2517 non-null   int64  
 1   Product Name    2517 non-null   string 
 2   Brand           2517 non-null   string 
 3   Color           2517 non-null   string 
 4   Unit Cost USD   2517 non-null   float64
 5   Unit Price USD  2517 non-null   float64
 6   SubcategoryKey  2517 non-null   int64  
 7   Subcategory     2517 non-null   string 
 8   CategoryKey     2517 non-null   int64  
 9   Category        2517 non-null   string 
dtypes: float64(2), int64(3), string(5)
memory usage: 196.8 KB
None


In [49]:
#push into pg sql , Products_cleaned data
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import create_engine

try:
    # Connect to PostgreSQL
    connection = psycopg2.connect(
        host="localhost",
        port="5432",
        database="Global_Electronics_Data",
        user="postgres",
        password="sample12"
    )
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cursor = connection.cursor()

    # Check if table exists
    cursor.execute("""
        SELECT EXISTS (
            SELECT FROM information_schema.tables 
            WHERE table_name = 'Products'
        );
    """)
    table_exists = cursor.fetchone()[0]

    if not table_exists:
        print("Table 'Products' does not exist. Creating it now...")
        cursor.execute("""
            CREATE TABLE Products (
                ProductKey INT PRIMARY KEY,
                Product_Name VARCHAR(255),
                Brand VARCHAR(255),
                Color VARCHAR(255),
                Unit_Cost_USD NUMERIC(10,2),
                Unit_Price_USD NUMERIC(10,2),
                SubcategoryKey INT,
                Subcategory VARCHAR(255),
                CategoryKey INT,
                Category VARCHAR(255)
            );
        """)
        print("Table created successfully.")

    # Insert data - using parameterized query
    for index, row in Products_cleaned.iterrows():
        cursor.execute("""
            INSERT INTO Products (
                ProductKey, Product_Name, Brand, Color,
                Unit_Cost_USD, Unit_Price_USD,
                SubcategoryKey, Subcategory,
                CategoryKey, Category
            ) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['ProductKey'], row['Product Name'], row['Brand'], row['Color'],
            row['Unit Cost USD'], row['Unit Price USD'],
            row['SubcategoryKey'], row['Subcategory'],
            row['CategoryKey'], row['Category']
        ))

    connection.commit()
    print(f"Successfully inserted {len(Products_cleaned)} records into Products table.")

except Exception as e:
    print(f"An error occurred while interacting with the database: {e}")
    if connection:
        connection.rollback()

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()

Table 'Products' does not exist. Creating it now...
Table created successfully.
Successfully inserted 2517 records into Products table.


*****Stores*****

In [1]:
# loading Stores.csv

import pandas as pd

file_path = r"C:/Users/USER/Desktop/DataSpark_Illuminating_Insights_for_Global_Electronics/Stores.csv"

# Method 1: Try common encodings automatically
encodings = ['utf-8', 'latin1', 'cp1252', 'ISO-8859-1']

for encoding in encodings:
    try:
        df_Stores = pd.read_csv(file_path, encoding=encoding)
        print(f"Successfully loaded with {encoding} encoding!")
        break
    except UnicodeDecodeError:
        continue

# Method 2: Manual encoding specification (if automatic fails)
# df_Stores = pd.read_csv(file_path, encoding='latin1')  # Most reliable fallback

# Verify the DataFrame
print("\nFirst 5 rows of df_Stores:")
print(df_Stores.head())

print("\nDataFrame info:")
print(df_Stores.info())

Successfully loaded with utf-8 encoding!

First 5 rows of df_Stores:
   StoreKey    Country                         State  Square Meters  Open Date
0         1  Australia  Australian Capital Territory          595.0   1/1/2008
1         2  Australia            Northern Territory          665.0  1/12/2008
2         3  Australia               South Australia         2000.0   1/7/2012
3         4  Australia                      Tasmania         2000.0   1/1/2010
4         5  Australia                      Victoria         2000.0  12/9/2015

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   StoreKey       67 non-null     int64  
 1   Country        67 non-null     object 
 2   State          67 non-null     object 
 3   Square Meters  66 non-null     float64
 4   Open Date      67 non-null     object 
dtypes: float64(1), int64(1), 

In [18]:
pd.set_option('display.max_rows', None)
print(df_Stores)

    StoreKey         Country                         State  Square Meters  \
0          1       Australia  Australian Capital Territory          595.0   
1          2       Australia            Northern Territory          665.0   
2          3       Australia               South Australia         2000.0   
3          4       Australia                      Tasmania         2000.0   
4          5       Australia                      Victoria         2000.0   
5          6       Australia             Western Australia         2000.0   
6          7          Canada                 New Brunswick         1105.0   
7          8          Canada     Newfoundland and Labrador         2105.0   
8          9          Canada         Northwest Territories         1500.0   
9         10          Canada                       Nunavut         1210.0   
10        11          Canada                         Yukon         1210.0   
11        12          France               Basse-Normandie          350.0   

In [2]:
# Convert categorical/text columns to string
df_Stores["Country"] = df_Stores["Country"].astype("string")
df_Stores["State"] = df_Stores["State"].astype("string")

# Convert numeric columns and replace NaN with 0
df_Stores["Square Meters"] = df_Stores["Square Meters"].astype(float).fillna(0)

# Convert date column to datetime
df_Stores["Open Date"] = pd.to_datetime(df_Stores["Open Date"], format="%m/%d/%Y", errors="coerce")

# Verify the changes
print(df_Stores.dtypes)

print("\nDataFrame info:")
print(df_Stores.info())

StoreKey                  int64
Country          string[python]
State            string[python]
Square Meters           float64
Open Date        datetime64[ns]
dtype: object

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   StoreKey       67 non-null     int64         
 1   Country        67 non-null     string        
 2   State          67 non-null     string        
 3   Square Meters  67 non-null     float64       
 4   Open Date      67 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), string(2)
memory usage: 2.7 KB
None


In [20]:
print(df_Stores.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   StoreKey       67 non-null     int64         
 1   Country        67 non-null     string        
 2   State          67 non-null     string        
 3   Square Meters  67 non-null     float64       
 4   Open Date      67 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), string(2)
memory usage: 2.7 KB
None


In [3]:

Stores_cleaned=df_Stores

In [4]:
Stores_cleaned.to_csv('Stores_cleaned.csv', index=False)
df_Customers_cleaned = pd.read_csv('Stores_cleaned.csv')

In [22]:
print(Stores_cleaned["Open Date"])#.unique())

0    2008-01-01
1    2008-01-12
2    2012-01-07
3    2010-01-01
4    2015-12-09
5    2010-01-01
6    2007-05-07
7    2014-07-02
8    2005-03-04
9    2015-04-04
10   2009-06-03
11   2012-06-06
12   2013-06-07
13   2009-12-15
14   2015-01-01
15   2010-06-03
16   2007-07-08
17   2012-08-08
18   2015-04-04
19   2012-12-15
20   2018-06-03
21   2008-03-06
22   2010-01-01
23   2012-12-15
24   2010-01-01
25   2019-03-05
26   2008-08-08
27   2012-12-15
28   2008-01-01
29   2008-01-12
30   2012-01-07
31   2010-01-01
32   2015-12-09
33   2010-01-01
34   2007-05-07
35   2014-07-02
36   2005-03-04
37   2015-04-04
38   2009-06-03
39   2012-06-06
40   2013-06-07
41   2009-12-15
42   2015-01-01
43   2010-06-03
44   2007-07-08
45   2012-08-08
46   2015-04-04
47   2012-12-15
48   2018-06-03
49   2008-03-06
50   2010-01-01
51   2009-06-03
52   2012-06-06
53   2013-06-07
54   2009-12-15
55   2015-01-01
56   2010-06-03
57   2007-07-08
58   2012-08-08
59   2005-04-04
60   2012-12-15
61   2018-06-03
62   200

In [16]:
import pandas as pd

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Now print the column
print(Stores_cleaned)#["Open Date"])

    StoreKey         Country                         State  Square Meters  \
0          1       Australia  Australian Capital Territory          595.0   
1          2       Australia            Northern Territory          665.0   
2          3       Australia               South Australia         2000.0   
3          4       Australia                      Tasmania         2000.0   
4          5       Australia                      Victoria         2000.0   
5          6       Australia             Western Australia         2000.0   
6          7          Canada                 New Brunswick         1105.0   
7          8          Canada     Newfoundland and Labrador         2105.0   
8          9          Canada         Northwest Territories         1500.0   
9         10          Canada                       Nunavut         1210.0   
10        11          Canada                         Yukon         1210.0   
11        12          France               Basse-Normandie          350.0   

In [24]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import create_engine

try:
    # Connect to PostgreSQL
    connection = psycopg2.connect(
        host="localhost",
        port="5432",
        database="Global_Electronics_Data",
        user="postgres",
        password="sample12"
    )
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cursor = connection.cursor()

    # Check if table exists
    cursor.execute("""
        SELECT EXISTS (
            SELECT FROM information_schema.tables 
            WHERE table_name = 'stores'
        );
    """)
    table_exists = cursor.fetchone()[0]

    if not table_exists:
        print("Table 'stores' does not exist. Creating it now...")
        cursor.execute("""
            CREATE TABLE stores (
                StoreKey INTEGER PRIMARY KEY,
                Country VARCHAR(255),
                State VARCHAR(255),
                Square_Meters NUMERIC(10,2),
                Open_Date DATE
            );
        """)
        print("Table created successfully.")

    # Insert data - using parameterized query
    for index, row in Stores_cleaned.iterrows():
        cursor.execute("""
            INSERT INTO stores (StoreKey, Country, State, Square_Meters, Open_Date)
            VALUES (%s, %s, %s, %s, %s)
        """, (
            row['StoreKey'],
            row['Country'],
            row['State'],
            row['Square Meters'],
            row['Open Date']
        ))

    connection.commit()
    print(f"Successfully inserted {len(Stores_cleaned)} records into stores table.")

except Exception as e:
    print(f"An error occurred while interacting with the database: {e}")
    if connection:
        connection.rollback()

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()

Table 'stores' does not exist. Creating it now...
Table created successfully.
Successfully inserted 67 records into stores table.


***Exchange_Rates***

In [5]:
# loading Exchange_Rates.csv

import pandas as pd

file_path = r"C:/Users/USER/Desktop/DataSpark_Illuminating_Insights_for_Global_Electronics/Exchange_Rates.csv"

# Method 1: Try common encodings automatically
encodings = ['utf-8', 'latin1', 'cp1252', 'ISO-8859-1']

for encoding in encodings:
    try:
        df_Exchange_Rates = pd.read_csv(file_path, encoding=encoding)
        print(f"Successfully loaded with {encoding} encoding!")
        break
    except UnicodeDecodeError:
        continue

# Method 2: Manual encoding specification (if automatic fails)
# df_Stores = pd.readf_Exchange_Ratesd_csv(file_path, encoding='latin1')  # Most reliable fallback

# Verify the DataFrame
print("\nFirst 5 rows of Exchange_Rates:")
print(df_Exchange_Rates.head())

print("\nDataFrame info:")
print(df_Exchange_Rates.info())

Successfully loaded with utf-8 encoding!

First 5 rows of Exchange_Rates:
       Date Currency  Exchange
0  1/1/2015      USD    1.0000
1  1/1/2015      CAD    1.1583
2  1/1/2015      AUD    1.2214
3  1/1/2015      EUR    0.8237
4  1/1/2015      GBP    0.6415

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11215 entries, 0 to 11214
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      11215 non-null  object 
 1   Currency  11215 non-null  object 
 2   Exchange  11215 non-null  float64
dtypes: float64(1), object(2)
memory usage: 263.0+ KB
None


In [29]:
import pandas as pd

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Now print the column
print(df_Exchange_Rates)

            Date Currency  Exchange
0     2015-01-01      USD    1.0000
1     2015-01-01      CAD    1.1583
2     2015-01-01      AUD    1.2214
3     2015-01-01      EUR    0.8237
4     2015-01-01      GBP    0.6415
5     2015-01-02      USD    1.0000
6     2015-01-02      CAD    1.1682
7     2015-01-02      AUD    1.2323
8     2015-01-02      EUR    0.8304
9     2015-01-02      GBP    0.6477
10    2015-01-03      USD    1.0000
11    2015-01-03      CAD    1.1682
12    2015-01-03      AUD    1.2323
13    2015-01-03      EUR    0.8304
14    2015-01-03      GBP    0.6477
15    2015-01-04      USD    1.0000
16    2015-01-04      CAD    1.1682
17    2015-01-04      AUD    1.2323
18    2015-01-04      EUR    0.8304
19    2015-01-04      GBP    0.6477
20    2015-01-05      USD    1.0000
21    2015-01-05      CAD    1.1784
22    2015-01-05      AUD    1.2384
23    2015-01-05      EUR    0.8393
24    2015-01-05      GBP    0.6569
25    2015-01-06      USD    1.0000
26    2015-01-06      CAD   

In [6]:
# Convert categorical/text columns to string
df_Exchange_Rates["Currency"] = df_Exchange_Rates["Currency"].astype("string")


# Convert numeric columns and replace NaN with 0
df_Exchange_Rates["Exchange"] = df_Exchange_Rates["Exchange"].astype(float).fillna(0)

# Convert date column to datetime
df_Exchange_Rates["Date"] = pd.to_datetime(df_Exchange_Rates["Date"], format="%m/%d/%Y", errors="coerce")

# Verify the changes
print(df_Exchange_Rates.dtypes)

print("\nDataFrame info:")
print(df_Exchange_Rates.info())

Date        datetime64[ns]
Currency    string[python]
Exchange           float64
dtype: object

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11215 entries, 0 to 11214
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      11215 non-null  datetime64[ns]
 1   Currency  11215 non-null  string        
 2   Exchange  11215 non-null  float64       
dtypes: datetime64[ns](1), float64(1), string(1)
memory usage: 263.0 KB
None


In [7]:
Exchange_Rates_cleaned=df_Exchange_Rates

In [8]:
Exchange_Rates_cleaned.to_csv('Exchange_Rates_cleaned.csv', index=False)
Exchange_Rates_cleaned = pd.read_csv('Exchange_Rates_cleaned.csv')

In [30]:
# push the data of exchange rates
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import create_engine

try:
    # Connect to PostgreSQL
    connection = psycopg2.connect(
        host="localhost",
        port="5432",
        database="Global_Electronics_Data",
        user="postgres",
        password="sample12"
    )
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cursor = connection.cursor()

    # Check if table exists
    cursor.execute("""
        SELECT EXISTS (
            SELECT FROM information_schema.tables 
            WHERE table_name = 'Exchange_Rates'
        );
    """)
    table_exists = cursor.fetchone()[0]

    if not table_exists:
        print("Table 'Exchange_Rates' does not exist. Creating it now...")
        cursor.execute("""
            CREATE TABLE Exchange_Rates (
                Date DATE,
                Currency VARCHAR(255),
                Exchange NUMERIC(10,4)  -- Changed to 4 decimal places
            );
        """)
        print("Table created successfully.")
    else:
        # If table exists, modify the Exchange column to 4 decimal places
        cursor.execute("""
            ALTER TABLE Exchange_Rates 
            ALTER COLUMN Exchange TYPE NUMERIC(10,4);
        """)
        print("Updated 'Exchange' column to 4 decimal places.")

    # Insert data (corrected to insert into Exchange_Rates)
    for index, row in Exchange_Rates_cleaned.iterrows():
        cursor.execute("""
            INSERT INTO Exchange_Rates (Date, Currency, Exchange)
            VALUES (%s, %s, %s)
        """, (
            row['Date'],
            row['Currency'],
            row['Exchange']
        ))

    connection.commit()
    print(f"Successfully inserted {len(Exchange_Rates_cleaned)} records into Exchange_Rates table.")

except Exception as e:
    print(f"An error occurred while interacting with the database: {e}")
    if connection:
        connection.rollback()

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()

Table 'Exchange_Rates' does not exist. Creating it now...
Table created successfully.
Successfully inserted 11215 records into Exchange_Rates table.


***Sales***

In [9]:
# loading Sales.csv

import pandas as pd

file_path = r"C:/Users/USER/Desktop/DataSpark_Illuminating_Insights_for_Global_Electronics/Sales.csv"

# Method 1: Try common encodings automatically
encodings = ['utf-8', 'latin1', 'cp1252', 'ISO-8859-1']

for encoding in encodings:
    try:
        df_Sales = pd.read_csv(file_path, encoding=encoding)
        print(f"Successfully loaded with {encoding} encoding!")
        break
    except UnicodeDecodeError:
        continue

# Method 2: Manual encoding specification (if automatic fails)
# df_Products = pd.read_csv(file_path, encoding='latin1')  # Most reliable fallback

# Verify the DataFrame
print("\nFirst 5 rows of df_Sales:")
print(df_Sales.head())

print("\nDataFrame info:")
print(df_Sales.info())

Successfully loaded with utf-8 encoding!

First 5 rows of df_Sales:
   Order Number  Line Item Order Date Delivery Date  CustomerKey  StoreKey  \
0        366000          1   1/1/2016           NaN       265598        10   
1        366001          1   1/1/2016     1/13/2016      1269051         0   
2        366001          2   1/1/2016     1/13/2016      1269051         0   
3        366002          1   1/1/2016     1/12/2016       266019         0   
4        366002          2   1/1/2016     1/12/2016       266019         0   

   ProductKey  Quantity Currency Code  
0        1304         1           CAD  
1        1048         2           USD  
2        2007         1           USD  
3        1106         7           CAD  
4         373         1           CAD  

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Order N

In [ ]:
import pandas as pd

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Now print the column
print(df_Sales["Delivery Date"])

In [37]:
# Convert date columns to datetime
df_Sales["Order Date"] = pd.to_datetime(df_Sales["Order Date"], format="%m/%d/%Y", errors="coerce")
# df_Sales["Delivery Date"] = pd.to_datetime(df_Sales["Delivery Date"], format="%m/%d/%Y", errors="coerce")
# Optional: Fill missing Delivery Dates (e.g., assume delivery after 7 days)
df_Sales["Delivery Date"] = df_Sales["Delivery Date"].fillna(df_Sales["Order Date"] + pd.Timedelta(days=7))

# Convert categorical/text columns to string
df_Sales["Currency Code"] = df_Sales["Currency Code"].astype("string")

# Ensure numeric columns are correct (if needed)
# (Assuming these should stay as integers; adjust if floats are needed)
numeric_cols = ["Order Number", "Line Item", "CustomerKey", "StoreKey", "ProductKey", "Quantity"]
df_Sales[numeric_cols] = df_Sales[numeric_cols].astype("int64")  # or "float64" if decimals exist

# Verify the changes
print(df_Sales.dtypes)

Order Number              int64
Line Item                 int64
Order Date       datetime64[ns]
Delivery Date            object
CustomerKey               int64
StoreKey                  int64
ProductKey                int64
Quantity                  int64
Currency Code    string[python]
dtype: object


In [10]:
import pandas as pd

# Step 1: Convert both columns to datetime FIRST
df_Sales["Order Date"] = pd.to_datetime(df_Sales["Order Date"], format="%m/%d/%Y", errors="coerce")
df_Sales["Delivery Date"] = pd.to_datetime(df_Sales["Delivery Date"], format="%m/%d/%Y", errors="coerce")

# Step 2: Fill NaN in Delivery Date (Order Date + 7 days)
delivery_date_changes = df_Sales["Delivery Date"].fillna(
    df_Sales["Order Date"] + pd.Timedelta(days=7)
)

# Step 3: Assign back to the DataFrame (optional, if you want to modify the original)
df_Sales["Delivery Date"] = delivery_date_changes

# Optional: Format as "%m/%d/%Y" strings (only for display/export)
df_Sales["Delivery Date"] = df_Sales["Delivery Date"].dt.strftime("%m/%d/%Y")

# Convert categorical/text columns to string
df_Sales["Currency Code"] = df_Sales["Currency Code"].astype("string")

# Ensure numeric columns are correct (if needed)
# (Assuming these should stay as integers; adjust if floats are needed)
numeric_cols = ["Order Number", "Line Item", "CustomerKey", "StoreKey", "ProductKey", "Quantity"]
df_Sales[numeric_cols] = df_Sales[numeric_cols].astype("int64")  # or "float64" if decimals exist

# Verify the changes
print(df_Sales.dtypes)

# Verify
print(df_Sales[["Order Date", "Delivery Date"]].head())

Order Number              int64
Line Item                 int64
Order Date       datetime64[ns]
Delivery Date            object
CustomerKey               int64
StoreKey                  int64
ProductKey                int64
Quantity                  int64
Currency Code    string[python]
dtype: object
  Order Date Delivery Date
0 2016-01-01    01/08/2016
1 2016-01-01    01/13/2016
2 2016-01-01    01/13/2016
3 2016-01-01    01/12/2016
4 2016-01-01    01/12/2016


In [11]:
import pandas as pd

# Step 1: Convert to datetime (handles "%m/%d/%Y" format)
df_Sales["Order Date"] = pd.to_datetime(df_Sales["Order Date"], format="%m/%d/%Y", errors="coerce")
df_Sales["Delivery Date"] = pd.to_datetime(df_Sales["Delivery Date"], format="%m/%d/%Y", errors="coerce")

# Step 2: Fill NaN in Delivery Date (Order Date + 7 days)
df_Sales["Delivery Date"] = df_Sales["Delivery Date"].fillna(
    df_Sales["Order Date"] + pd.Timedelta(days=7)
)

In [12]:
# Verify the changes
print(df_Sales.dtypes)

Order Number              int64
Line Item                 int64
Order Date       datetime64[ns]
Delivery Date    datetime64[ns]
CustomerKey               int64
StoreKey                  int64
ProductKey                int64
Quantity                  int64
Currency Code    string[python]
dtype: object


In [13]:
# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Now print the column
print(df_Sales[["Order Date", "Delivery Date"]].head())

  Order Date Delivery Date
0 2016-01-01    2016-01-08
1 2016-01-01    2016-01-13
2 2016-01-01    2016-01-13
3 2016-01-01    2016-01-12
4 2016-01-01    2016-01-12


In [14]:
Sales_cleaned=df_Sales

In [15]:
Sales_cleaned.to_csv('Sales_cleaned.csv', index=False)
Sales_cleaned = pd.read_csv('Sales_cleaned.csv')

In [47]:
print(Sales_cleaned.columns)


Index(['Order Number', 'Line Item', 'Order Date', 'Delivery Date',
       'CustomerKey', 'StoreKey', 'ProductKey', 'Quantity', 'Currency Code'],
      dtype='object')


In [56]:
# push the data of Sales tables
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import create_engine

try:
    # Connect to PostgreSQL
    connection = psycopg2.connect(
        host="localhost",
        port="5432",
        database="Global_Electronics_Data",
        user="postgres",
        password="sample12"
    )
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cursor = connection.cursor()

    # Check if table exists
    cursor.execute("""
        SELECT EXISTS (
            SELECT FROM information_schema.tables 
            WHERE table_name = 'Sales'
        );
    """)
    table_exists = cursor.fetchone()[0]

    if not table_exists:
        print("Table 'Sales' does not exist. Creating it now...")
        cursor.execute("""
            CREATE TABLE Sales (
                Order_Number INT,
                Line_Item INT,
                Order_Date DATE NOT NULL,
                Delivery_Date DATE,
                customerkey INT NOT NULL,
                storekey INT,
                productkey INT NOT NULL,
                Quantity INT NOT NULL ,
                Currency_Code VARCHAR(5),
                PRIMARY KEY (order_number, line_item),
                FOREIGN KEY (customerkey) REFERENCES customers(customerkey),
                FOREIGN KEY (productkey) REFERENCES products(productkey),
                FOREIGN KEY (storekey) REFERENCES stores(storekey)
            
            );
        """)
        print("Table created successfully.")
    else:
        # If table exists, modify the Exchange column to 4 decimal places
        cursor.execute("""
            ALTER TABLE Sales 
            ALTER COLUMN Sales TYPE NUMERIC(10,4);
        """)
        print("Updated 'Sales' column to 4 decimal places.")

    # Insert data (corrected to insert into Sales)
    for index, row in Sales_cleaned.iterrows():
        cursor.execute("""
            INSERT INTO Sales (Order_Number, Line_Item, Order_Date, Delivery_Date,
       CustomerKey, storekey, productkey, Quantity, Currency_Code)
            VALUES (%s, %s, %s,%s, %s, %s,%s, %s, %s)
        """, (
            row['Order Number'],
            row['Line Item'],
            row['Order Date'],
            row['Delivery Date'],
            row['CustomerKey'],
            row['StoreKey'],
            row['ProductKey'],
            row['Quantity'],
            row['Currency Code']
        ))

    connection.commit()
    print(f"Successfully inserted {len(Sales_cleaned)} records into Sales_cleaned table.")

except Exception as e:
    print(f"An error occurred while interacting with the database: {e}")
    if connection:
        connection.rollback()

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()

Table 'Sales' does not exist. Creating it now...
Table created successfully.
Successfully inserted 62884 records into Sales_cleaned table.


In [50]:
# Push the data of Sales table with foreign key constraints
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from psycopg2.extras import execute_batch
import sys

def push_sales_data(Sales_cleaned):
    connection = None
    cursor = None
    inserted_count = 0
    
    try:
        # Connect to PostgreSQL
        connection = psycopg2.connect(
            host="localhost",
            port="5432",
            database="Global_Electronics_Data",
            user="postgres",
            password="sample12"
        )
        connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        cursor = connection.cursor()

        # 1. Verify table exists with correct structure
        cursor.execute("""
            SELECT EXISTS (
                SELECT FROM information_schema.tables 
                WHERE table_name = 'sales'
            );
        """)
        table_exists = cursor.fetchone()[0]

        if not table_exists:
            print("Creating sales table...")
            cursor.execute("""
                CREATE TABLE sales (
                    order_number INT,
                    line_item INT,
                    order_date DATE NOT NULL,
                    delivery_date DATE,
                    customerkey INT NOT NULL,
                    storekey INT,
                    productkey INT NOT NULL,
                    quantity INT NOT NULL CHECK (quantity > 0),
                    currency_code VARCHAR(3),
                    PRIMARY KEY (order_number, line_item),
                    FOREIGN KEY (customerkey) REFERENCES customers(customerkey),
                    FOREIGN KEY (productkey) REFERENCES products(productkey),
                    FOREIGN KEY (storekey) REFERENCES stores(storekey)
                );
            """)
            print("Sales table created successfully.")

        # 2. Prepare data for insertion
        records = []
        for _, row in Sales_cleaned.iterrows():
            records.append((
                row['Order Number'],
                row['Line Item'],
                row['Order Date'],
                row['Delivery Date'],
                row['CustomerKey'],
                row['StoreKey'],
                row['ProductKey'],
                row['Quantity'],
                row['Currency Code']
            ))

        # 3. Insert data with progress feedback
        print(f"Starting insertion of {len(records)} records...")
        
        execute_batch(
            cursor,
            """
            INSERT INTO sales (
                order_number, line_item, order_date, delivery_date,
                customerkey, storekey, productkey, quantity, currency_code
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (order_number, line_item) DO NOTHING
            """,
            records,
            page_size=100
        )
        
        # Get actual inserted count
        cursor.execute("SELECT COUNT(*) FROM sales WHERE order_date >= %s", 
                      (Sales_cleaned['Order Date'].min(),))
        inserted_count = cursor.fetchone()[0]
        
        connection.commit()
        print(f"\nSuccessfully inserted {inserted_count} new records into sales table.")
        print(f"Skipped {len(records) - inserted_count} duplicate records.")

        return inserted_count

    except psycopg2.IntegrityError as e:
        print("\nERROR: Foreign key violation detected!", file=sys.stderr)
        print("Please verify these reference tables exist and contain all required keys:", file=sys.stderr)
        print("- customers (for customerkey)", file=sys.stderr)
        print("- products (for productkey)", file=sys.stderr)
        print("- stores (for storekey)", file=sys.stderr)
        print(f"Detailed error: {e}", file=sys.stderr)
        if connection:
            connection.rollback()
        return 0

    except Exception as e:
        print(f"\nERROR: Database operation failed: {e}", file=sys.stderr)
        if connection:
            connection.rollback()
        return 0

    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()
        print("Database connection closed.")

# Usage example:
# inserted = push_sales_data(Sales_cleaned)
# if inserted > 0:
#     print("Data load successful!")